In [14]:
!pip install --quiet SPARQLWrapper
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [15]:
import anvil.server


In [16]:
anvil.server.connect("server_WIONFUIB7XXXWRJEH27B4TD2-DERCMR7TIJFOJCY3")

In [17]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import pandas as pd

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [18]:
# get list of common ingredients

ing_qstr = """SELECT DISTINCT ?ing ?ingLabel
WHERE
{
    ?food wdt:P279 wd:Q2095;
        wdt:P527 ?ing.
SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}"""
sparql.setQuery(ing_qstr)
sparql.setReturnFormat(JSON)
ings = sparql.query().convert()

In [19]:
ings_df = pd.io.json.json_normalize(ings['results']['bindings'])
ings_df = ings_df[['ing.value', 'ingLabel.value']]
ings_df.columns = ['qid', 'name']
ings_df['qid'] = ings_df['qid'].str.split('/').str[-1]

<ipython-input-19-c4cd6b13ce46>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  ings_df = pd.io.json.json_normalize(ings['results']['bindings'])


In [20]:
ings_dct = dict(zip(ings_df['name'], ings_df['qid']))

In [21]:
''' get closest match of entered ingredient and available list of ingredients '''

' get closest match of entered ingredient and available list of ingredients '

In [22]:
def wiki_id(name):
  if name not in ings_dct:
    for dct_name, qid in ings_dct.items():
      if (name in dct_name) or (dct_name in name):
        return qid
    return None
  else:
    return ings_dct[name]

In [23]:
def form_query(required_ing, forbidden_ing):
  query = [
    'SELECT DISTINCT ?food ?ingredient ?foodLabel ?ingredientLabel',
    'WHERE',
    '{'
  ]

  temp_union_query = [f'{{?food wdt:P527 wd:{x}.}}' for x in required_ing]
  for e in temp_union_query:
    query.append(e)
    query.append('UNION')
  query.pop()

  # required_ingredient_query = [f'wdt:P527 wd:{x};' for x in required_ing[1:]]
  # query.extend(required_ingredient_query)
  # query.append()

  if len(forbidden_ing)>0:
    minus_query = [f'MINUS {{ ?food wdt:P527 wd:{x}. }}' for x in forbidden_ing]

    query.extend(minus_query)

  query_end = [
      '?food wdt:P527 ?ingredient.',
      'SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }',
      '}',
  ]
  query.extend(query_end)
  return query

In [24]:

def get_results(required_ing, forbidden_ing, return_query=False):
  required_ing = [wiki_id(x) for x in required_ing if wiki_id(x) is not None]
  forbidden_ing = [wiki_id(x) for x in forbidden_ing if wiki_id(x) is not None]

  formatted_query = '\n'.join(form_query(required_ing, forbidden_ing))

  sparql.setQuery(' '.join(form_query(required_ing, forbidden_ing)))
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()

  df = pd.json_normalize(results['results']['bindings'])
  df = df[['foodLabel.value', 'ingredientLabel.value']]
  df.columns = ['Food', 'Ingredients']
  df = df.sort_values(by=['Food', 'Ingredients'])
  df = df.sort_values(by=['Food', 'Ingredients']).reset_index(drop=True)

  return df, formatted_query

In [25]:
@anvil.server.callable
def fetch_recipes(required_ingredients,prohibited_ingredients):
  required = required_ingredients.split()
  forbidden = prohibited_ingredients.split()

  results, query = get_results(required, forbidden)

  if results.empty:
    val = False
  else:
    val = True

  return results.to_markdown(), val


In [26]:
anvil.server.wait_forever()

KeyboardInterrupt: ignored

In [ ]:
# required = ['tomato', 'bread']
# forbidden = ['mayonnaise', 'onion']

# results, query = get_results(required, forbidden)

In [ ]:
results